In [1]:
import pandas as pd
import numpy as np

In [6]:
raw = pd.read_csv('../data/qgis.csv', index_col=0)

In [8]:
raw.head()

,SpaceID,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,StreetCleaning,LatLng,Lat,Lng
0,VS336,4900 SUNSET BL,Single-Space,FLAT,$1.00,2HR,"TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat",NaN,"(34.098024, -118.296664)",34.098024,-118.296664
1,ED396,700 E PICO BLVD,Single-Space,TOD,$0.5 - $1,2HR,8A-6P Mon-Sat,NaN,"(34.032054, -118.252521)",34.032054,-118.252521
2,CB211A,1300 S GRAND AVE,Single-Space,FLAT,$0.50,4HR,8A-8P Mon-Sat,NaN,"(34.038125, -118.264675)",34.038125,-118.264675
3,SL408,3100 SUNSET BL,Single-Space,FLAT,$1.00,2HR,8A-8P Mon-Sat,NaN,"(34.084343, -118.274463)",34.084343,-118.274463
4,VN1,7001 VAN NUYS BL,Single-Space,FLAT,$1.00,2HR,8A-8P Mon-Sat,NaN,"(34.198035, -118.44891)",34.198035,-118.448910


In [22]:
grouped = raw.groupby(['BlockFace', 'MeterType','RateType','RateRange','MeteredTimeLimit','ParkingPolicy'])\
    .agg({'LatLng': lambda x : ', '.join(x)})
grouped.reset_index(inplace=True)

In [23]:
grouped['group'] = grouped.index

In [74]:
def rm_parentheses(s: str):
    return s.replace('(', '').replace(')', '').replace(' ', '')

def split_n(s, sep, n):
    s = s.split(sep)
    return [s[i:i+n] for i in range(0, len(s), n)]

def split_latlng(latlng: str):
    stripped = rm_parentheses(latlng)
    coords = split_n(stripped, ',', 2)
    return coords

In [76]:
grouped['coord_list'] = grouped['LatLng'].apply(split_latlng)

In [79]:
grouped = grouped[['group', 'coord_list']].explode('coord_list').reset_index(drop=True)
grouped['order'] = grouped.index

In [81]:
grouped['lat'] = [coord[0] for coord in grouped.coord_list]
grouped['long'] = [coord[1] for coord in grouped.coord_list]

In [83]:
df = grouped[['group', 'order', 'lat', 'long']]

In [85]:
df.to_csv('../data/grouped_qgis.csv')